### Manager

#### Purpose:
1. Setup the log table
2. Setup the error table
3. Setup the company list and set the Redis Value
4. Control the Sync schedule (When to set the DataOn variable)
5. Subscribe to the error subscription and log errors 

In [1]:
############# Imports ##################################
import redis
import json
import schedule
import time
from datetime import date, datetime
import os
import pandas as pd

import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy import MetaData
from sqlalchemy import Table
from sqlalchemy import Column
from sqlalchemy import Integer, String, DateTime

############# Redis Setup ##############################
#Connect to the DataStore
REDIS = redis.Redis(host='data_store')

#To setup the queue
queue = REDIS.pubsub()
#Subscribe to the queues one for the events and one for the log
queue.subscribe('event_queue')
queue.subscribe('log_queue')

In [2]:
############# Setup Database ###########################
# User = %env DB_USER
# password = %env DB_PWD
# dbname = %env DB_NAME

User = os.environ['DB_USER']
password = os.environ['DB_PWD']
dbname = os.environ['DB_NAME']

engine = create_engine('mysql+mysqlconnector://{}:{}@db:3306/{}'.format(User, password, dbname), echo=False)
conn = engine.connect()



#Check to see if the tables are created and if not, create them
meta = MetaData(engine)

#Create log table
if not engine.dialect.has_table(engine, 'log'):
    print('Log Table does not exist')
    print('Log Table being created....')
    #Time, Source, Current Count, Count Diff
    t1 = Table('log', meta,
                 Column('log_time', DateTime, default=datetime.utcnow),
                 Column('Source', String(30)),
                 Column('Current_Count', Integer),
                 Column('Count_Diff', Integer))
    t1.create()
else:
    print('Log Table Exists')

#Create event table
if not engine.dialect.has_table(engine, 'event'):
    print('Event Table does not exist')
    print('Event Table being created....')
    #Time, Source, Current Count, Count Diff
    t1 = Table('event', meta,
                 Column('event_time', DateTime, default=datetime.utcnow),
                 Column('Source', String(30)),
                 Column('Kind', String(30)),
                 Column('Message', String(8000)))
    t1.create()
else:
    print('Event Table Exists')


    

Log Table Exists
Event Table Exists


In [3]:
################## Define Functions ##############################

#Serialize datetime.
def json_serial(obj):
    """JSON serializer for objects not serializable by default json code"""

    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError ("Type %s not serializable" % type(obj))



#Create table object
meta = MetaData(engine,reflect=True)
log_table = meta.tables['log']

def database_log(log_data):
    #Need to log these items to a database.
    #Convert the data
    log_data = json.loads(log_data)
    print('------ Database Log Data --------')
    print(log_data)
    ins = log_table.insert().values(
            log_time = log_data['log_time'],
            Source = log_data['source'],
            Current_Count = log_data['current_count'],
            Count_Diff = log_data['count_diff']
               )
    conn.execute(ins)
    print('Logged Log Data')
    
#Create table object
#meta = MetaData(engine,reflect=True)
event_table = meta.tables['event']

def database_event(event_data):
    #Need to log these items to a database.
    event_data = json.loads(event_data)
    
    ins = event_table.insert().values(
            event_time = event_data['event_time'],
            Source = event_data['source'],
            Kind=event_data['kind'],
            Message=event_data['message']
               )
    conn.execute(ins)
    print('Logged Event Data')

#Event Kind Types
#Activity, Error
    
#Manager will send itself a message when it turns data on and off    
def send_event(source, kind, message):
    event_time = datetime.now()
    event_time = json_serial(event_time)
    event = {
            "event_time": event_time,
            "source": source,
            "kind" : kind,
            "message" : message
            }
    payload = json.dumps(event)
    REDIS.publish('event_queue', payload)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:14: SADeprecationWarning: reflect=True is deprecate; please use the reflect() method.
  


In [4]:
########### Set up Feature Flag #####################

#Setup file read and set state

def update_flags_from_file():
    feature_flags = pd.read_csv('feature_flag.csv', index_col='Feature')
    #Set the feture flags in REDIS
    REDIS.set('feature_flags', feature_flags.to_msgpack(compress='zlib'))
    
def get_feature_flag(feature):
    all_flags = pd.read_msgpack(REDIS.get("feature_flags"))
    
    try:
        return all_flags.get_value(feature, 'State')
        
    except:
        return 'Flag Not Found, not a valid feature flag'

In [5]:
########### Set up the company list #####################
companies = {
    "AAPL":"Apple",
    "FB":"Facebook",
    "GOOG":"Google Alphabet C",
    "GOOGL":"Google Alphabet A",
    "AMZN":"Amazon",
    "MSFT":"Microsoft",
    "BAC":"Bank of America",
    "BA":"Boeing",
    "NFLX":"Netflix",
    "JPM":"JPMorgan Chase",
    "TSLA":"Tesla",
    "CSCO":"Cisco Systems",
    "XOM":"Exxon Mobil",
    "WFC":"Wells Fargo",
    "V":"Visa",
    "JNJ":"Johnson & Johnson",
    "PFE" :"Pfizer",
    "INTC":"Intel",
    "HD":"Home Depot",
    "C":"Citigroup",
    "UTX":"United Technologies",
    "CMCSA":"Comcast",
    "TWTR":"Twitter",
    "GE":"General Electric",
    "UNH":"United Health Group",
    "PCLN":"Priceline Group",
    "BABA":"Alibaba Group",
    "NVDA":"NVIDIA"
    
    }
REDIS.set('companies', json.dumps(companies))

True

In [6]:
########### Setup the Data ON Flag #################
REDIS.set('Data_On',0)

#Create Explicit Data On/Off Functions
def DataOn():
    REDIS.set('Data_On',1)
    send_event('Manager', 'Activity', 'Data flag On')
    

def DataOff():
    REDIS.set('Data_On',0)
    send_event('Manager', 'Activity', 'Data flag Off')
    
# global past_twitter_count
# past_tweet_count = 0

In [7]:
#Turn on if starts during open times
now = datetime.now()

if now.hour >= 14 and now.hour < 21:
    if now.hour == 14 and now.minute >= 30:
        DataOn()
    elif now.hour > 14:
        DataOn()

In [8]:
########## Setup Schedules ######################
#Since the market isnt open all day, want to control when turn data fetch on and off

#Time in 24 hour clocks and UTC time
schedule.clear()
#Setup Data On/Off Schedule
schedule.every().monday.at("14:30").do(DataOn)
schedule.every().monday.at("21:00").do(DataOff)
schedule.every().tuesday.at("14:30").do(DataOn)
schedule.every().tuesday.at("21:00").do(DataOff)
schedule.every().wednesday.at("14:30").do(DataOn)
schedule.every().wednesday.at("21:00").do(DataOff)
schedule.every().thursday.at("14:30").do(DataOn)
schedule.every().thursday.at("21:00").do(DataOff)
schedule.every().friday.at("14:30").do(DataOn)
schedule.every().friday.at("21:00").do(DataOff)

#Read the Feature Flag
schedule.every(10).seconds.do(update_flags_from_file)


Every 10 seconds do update_flags_from_file() (last run: [never], next run: 2018-03-05 19:39:55)

In [9]:
########### Execute ############################

#Read Feature Flags at beginning
update_flags_from_file()

while True:
    schedule.run_pending()
    
    #May need to add in another while loop here. But we shall see after testing.
    
    next_message = queue.get_message()
    #next_message = json.loads(queue.get_message()['data'].decode())
    if next_message:
        print('------ REDIS Message -------')
        print(next_message)
        #Ignore the initial 1 or 2 that comes out of the queue.
        try:
            payload = next_message['data'].decode()
            #check which queue
            if next_message['channel'].decode() == 'event_queue':
                #Call database_event function to log to database
                database_event(payload)
                #Eventually can check the kind of event and do different action if error.
        
            if next_message['channel'].decode() == 'log_queue':
                #Call database_log function to log to database
                database_log(payload)
        
        except:
            pass
        
        
    
    time.sleep(1)

------ REDIS Message -------
{'type': 'subscribe', 'pattern': None, 'channel': b'event_queue', 'data': 1}
------ REDIS Message -------
{'type': 'subscribe', 'pattern': None, 'channel': b'log_queue', 'data': 2}
------ REDIS Message -------
{'type': 'message', 'pattern': None, 'channel': b'event_queue', 'data': b'{"event_time": "2018-03-05T19:39:43.675919", "source": "Manager", "kind": "Activity", "message": "Data flag On"}'}
Logged Event Data
------ REDIS Message -------
{'type': 'message', 'pattern': None, 'channel': b'event_queue', 'data': b'{"event_time": "2018-03-05T19:40:22.086349", "source": "Twitter", "kind": "Activity", "message": "Data Source Started"}'}
Logged Event Data
------ REDIS Message -------
{'type': 'message', 'pattern': None, 'channel': b'log_queue', 'data': b'{"log_time": "2018-03-05T19:40:52.151502", "source": "Twitter", "current_count": 1, "count_diff": 1}'}
------ Database Log Data --------
{'log_time': '2018-03-05T19:40:52.151502', 'source': 'Twitter', 'current_

KeyboardInterrupt: 

In [24]:
dict_test = {"log_time": "2018-02-05T00:37:45.578451", "source": "IEX", "current_count": 12, "count_diff": 3}

In [25]:
dict_test['log_time']

'2018-02-05T00:37:45.578451'